## Imports

In [1]:
import numpy as np
import tensorflow as tf

## Prepare the data

Our dummy dataset is just a pair of arrays xs and ys defined by the relationship  𝑦=2𝑥−1 . xs are the inputs while ys are the labels.

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Training the model

In [3]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))

[[18.97765]]


## Custom loss

In [4]:
def my_huber_loss(y_true, y_pred):
    error = y_true - y_pred
    threshold = 1
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [5]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss)
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))

[[18.491188]]


## Custom Loss with Threshold

In [6]:
def my_huber_loss_with_threshold(threshold):
    def my_huber_loss(y_true, y_pred):
        error = tf.abs(y_true - y_pred)
        is_small_error = error <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (error - 0.5 * threshold)
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return my_huber_loss

In [7]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1.4))
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))

[[18.617338]]


## Implement Custom Loss as a Class

In [8]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
    
    def __init__(self, threshold=1):
        super().__init__()
        self.threshold = threshold
        
    def call(self, y_true, y_pred):
        error = tf.abs(y_true - y_pred)
        is_small_error = error <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (error - 0.5 * self.threshold)
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [9]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=MyHuberLoss())
model.fit(xs, ys, epochs=500, verbose=0)

print(model.predict([10.0]))

[[18.601597]]
